* **RAW MLE**
* Simply ratio between number of times the $word_{i}$ occurred in a class and total number of words in the class. 
 
 $P(\text{word}_i \mid c) \;=\; \frac{\text{count}(\text{word}_i, c)}{\sum_{w} \text{count}(w, c)}$

* **SMOOTHED MLE**
* However, we add a single count for every word's count so that we do not get any zero occurrences between the classes. This is to ensure that all the words occurred at least once in all the classes. Because we have added 1 to the numerator (every single word's count is increased by 1). That is equal to vacb_size so, we must add that to the numerator (AKA Laplace Smoothing).
  
$P(\text{word}_i \mid c) \;=\; 
\frac{\text{count}(\text{word}_i, c) + 1}{
   \sum_{w} \text{count}(w, c) \;+\; \lvert V \rvert}$

* We know that $Posterior \propto Prior \times Likelihood$
* Prior is simply the ratio of number of samples in a class and total number of samples (typically training set)

 $P(c) \;=\; \frac{\text{count}(c)}{\sum_{c'} \text{count}(c')}$
 
 * Probability of a sentence belonging to a class (sentence is $w_1, w_2, \ldots, w_n$). The whole point of Naive bayes is that we simply take the products of these likelihoods as if they're independent (many times they're not!!). See the notebook at `naive-bayes-practice/naive-bayes-gaussian/naive_bayes_scratch.ipynb` for more on Naive Bayes.
   
 $P(c \mid w_1, w_2, \ldots, w_n)
\;\propto\;
P(c)
\;\prod_{i=1}^{n}
P\bigl(w_i \mid c\bigr)$


In [1]:
import re
import random
import math
from collections import defaultdict, Counter

# ---------------------------
# 1. A small SMS dataset
# ---------------------------
sample_sms_data = [
    ("Free entry in 2 a wkly comp to win FA Cup final tkts!!", "spam"),
    ("U dun say so early hor... U c already then say...", "ham"),
    ("Nah I don't think he goes to usf, he lives around here though", "ham"),
    ("Free tickets available, exclusive offer just for you!", "spam"),
    ("Win cash now!!! Just send WIN to 12345", "spam"),
    ("Hello, hope you are doing well my friend.", "ham")
]

# ---------------------------
# 2. Train/Test Split
# ---------------------------
def train_test_split(data, test_size=0.33, seed=42):
    """ Shuffle and split into train and test sets. """
    random.seed(seed)
    shuffled = data[:]
    random.shuffle(shuffled)
    split_point = int(len(shuffled) * (1 - test_size))
    return shuffled[:split_point], shuffled[split_point:]

train_data, test_data = train_test_split(sample_sms_data, test_size=0.33)


# ---------------------------
# 3. Preprocessing Functions
# ---------------------------
def preprocess_text(text):
    """
    - Lowercase
    - Remove non-alphanumeric (punctuation)
    - Tokenize (split on whitespace)
    """
    text = text.lower()
    # remove punctuation using regex
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # tokenize by whitespace
    tokens = text.split()
    return tokens

def build_vocabulary(dataset):
    """
    Build a vocabulary set of all unique words from the training data.
    """
    vocab = set()
    for sms, label in dataset:
        tokens = preprocess_text(sms)
        for t in tokens:
            vocab.add(t)
    return sorted(list(vocab))  # sorted list (optional)


# ---------------------------
# 4. Train a Multinomial NB Model
# ---------------------------
def train_multinomial_nb(train_data):
    """
    Train a Multinomial Naive Bayes:
      - Compute P(class) (priors)
      - Compute P(word | class) for each word
    Returns model parameters:
      - class_priors: dict -> {class_label: prior_probability}
      - word_counts: dict -> {class_label: {word: count}}
      - total_words_in_class: dict -> {class_label: total_count_of_words}
      - vocabulary
    """
    # Count how many docs per class
    class_counts = defaultdict(int)  # {spam: X, ham: Y}
    
    # Word counts per class: {spam: {word: count}, ham: {word: count}}
    word_counts = defaultdict(lambda: defaultdict(int))
    
    for sms, label in train_data:
        class_counts[label] += 1
        tokens = preprocess_text(sms)
        for token in tokens:
            word_counts[label][token] += 1

    # Compute total docs
    total_docs = sum(class_counts.values())
    
    # Compute priors: P(class)
    class_priors = {}
    for c in class_counts:
        class_priors[c] = class_counts[c] / total_docs

    # Build vocabulary from training data
    vocabulary = build_vocabulary(train_data)

    # Compute total words in each class
    total_words_in_class = {}
    for c in class_counts:
        total_words_in_class[c] = sum(word_counts[c].values())

    # Return a dictionary holding all necessary info
    model = {
        'class_priors': class_priors,
        'word_counts': dict(word_counts),
        'total_words_in_class': total_words_in_class,
        'vocabulary': vocabulary,
        'class_counts': class_counts
    }
    return model

def predict_class(model, text):
    """
    Predict class for a given text using the trained Multinomial NB model.
    We do:
       P(c | doc) ~ P(c) * Π [ P(word_i | c) ]
    Where P(word_i | c) = (count(word_i, c) + 1) / (total_words_in_class[c] + vocab_size)
    (We apply Laplace smoothing by adding 1).
    """
    tokens = preprocess_text(text)
    class_priors = model['class_priors']
    word_counts = model['word_counts']
    total_words_in_class = model['total_words_in_class']
    vocabulary = model['vocabulary']
    vocab_size = len(vocabulary)
    
    # Calculate posterior for each class
    scores = {}
    for c in class_priors:
        # Start with log(P(c))
        log_prob = math.log(class_priors[c])
        
        # For each word in the SMS
        for w in tokens:
            # Laplace smoothing
            count_wc = word_counts[c].get(w, 0)
            p_w_given_c = (count_wc + 1) / (total_words_in_class[c] + vocab_size)
            # use log probabilities to avoid underflow
            log_prob += math.log(p_w_given_c)
        
        scores[c] = log_prob
    
    # pick the class with highest log-prob
    predicted_class = max(scores, key=scores.get)
    return predicted_class

def evaluate(model, test_data):
    """
    Evaluate accuracy on test set.
    """
    correct = 0
    for sms, label in test_data:
        pred = predict_class(model, sms)
        if pred == label:
            correct += 1
    return correct / len(test_data)

# ---------------------------
# 5. Putting it all together
# ---------------------------
if __name__ == "__main__":
    # Train
    model = train_multinomial_nb(train_data)
    
    # Test
    accuracy = evaluate(model, test_data)
    
    # Print some results
    print("Training samples:", len(train_data))
    print("Test samples:", len(test_data))
    print("Vocabulary size:", len(model['vocabulary']))
    print(f"Accuracy on test set = {accuracy*100:.2f}%")

    # Demo prediction
    test_sms = "Hurry! Free cash prize if you call now"
    predicted_label = predict_class(model, test_sms)
    print(f"\nMessage: '{test_sms}' -> Predicted Label: {predicted_label}")


Training samples: 4
Test samples: 2
Vocabulary size: 34
Accuracy on test set = 50.00%

Message: 'Hurry! Free cash prize if you call now' -> Predicted Label: spam
